In [1]:
import re
import json
import requests
from bs4 import BeautifulSoup
import time
from concurrent.futures import ThreadPoolExecutor, wait
from datetime import datetime


today = time.strftime("%Y%m%d", time.localtime())


total_counter = 0
one_result_counter = 0
perfectly_matched_result_counter = 0
no_movie_data_counter = 0
more_results_counter = 0
no_data_counter = 0

movies_reviews_dict = {}

def movies_crawler(movie_title):    
    
    global total_counter
    total_counter += 1

    search_url = 'http://search.atmovies.com.tw/search/'
    headers = {'Referer':'http://search.atmovies.com.tw/search/'}
    form_data = {'type':'movie','search_term':movie_title}
    r = requests.post(search_url, data = form_data, headers = headers)
    s = BeautifulSoup(r.text, "html5lib")

    movie_value_dict = {}

    if s.select('blockquote > header') != []:
        
        result_type_list = []
        possible_title_ch_list = []
        possible_title_en_list = []
        possible_href_list = []
        
        possible_title_ch_nomoviedata_list = []
        possible_title_en_nomoviedata_list = [] 
        
        for i in range(len(s.select('blockquote > header'))):
            result_type = s.select('blockquote > header')[i].select('font')[0].text
            result_type_list.append(result_type)
            movie_title_xs = movie_title.replace(' ', '')
            
            movie_title_raw = s.select('blockquote > header')[i].select('a')[0].text
            movie_title_ch = movie_title_raw.split(" ")[0]
            if movie_title_ch not in possible_title_ch_nomoviedata_list:
                possible_title_ch_nomoviedata_list.append(movie_title_ch)
            
            movie_title_en = ''.join(movie_title_raw.split(" ")[1:])
            if movie_title_en not in possible_title_en_nomoviedata_list:
                possible_title_en_nomoviedata_list.append(movie_title_en)
                
            if s.select('blockquote > header')[i].select('font')[0].text == '電影' and int(s.select('blockquote > header')[i].select('font')[1].text) < 2018 : 

                movie_review_href = s.select('blockquote > header')[i].select('a')[0]['href']

                if len(s.select('blockquote > header')) == 1:
                    global one_result_counter
                    one_result_counter += 1

                    movie_value_dict['name'] = movie_title_ch
                    movie_value_dict['memo'] = 'only 1 result'
                    movie_value_dict['Ename'] = movie_title_en
                    movie_value_dict['href'] = movie_review_href
                    movies_reviews_dict[movie_title] = movie_value_dict
                    print('movie_title: %s, one_result_counter: %s'%(movie_title, one_result_counter))

                else:

                    if movie_title_xs == movie_title_en:
                        global perfectly_matched_result_counter
                        perfectly_matched_result_counter += 1
                        
                        movie_value_dict['name'] = movie_title_ch
                        movie_value_dict['memo'] = 'perfectly matched result'
                        movie_value_dict['Ename'] = movie_title_en
                        movie_value_dict['href'] = movie_review_href
                        movies_reviews_dict[movie_title] = movie_value_dict
                        print('movie_title: %s, perfectly_matched_result_counter: %s'%(movie_title, perfectly_matched_result_counter))

                        break

                    else:
                        possible_title_en_list.append(movie_title_en)
                        possible_title_ch_list.append(movie_title_ch)
                        possible_href_list.append(movie_review_href)                        
        
        if '電影' not in result_type_list: #或if not...in...
            global no_movie_data_counter
            no_movie_data_counter += 1
            
            movie_value_dict['name'] = possible_title_ch_nomoviedata_list
            movie_value_dict['memo'] = 'no movie data @atmovies'
            movie_value_dict['possible_title_en'] = possible_title_en_nomoviedata_list
            movies_reviews_dict[movie_title] = movie_value_dict
            print('movie_title: %s, no_movie_data_counter: %s'%(movie_title, no_movie_data_counter))
            
        elif movies_reviews_dict == {}: #不穩定
            global more_results_counter
            more_results_counter += 1
            
            movie_value_dict['name'] = possible_title_ch_list
            movie_value_dict['memo'] = 'need more detailed codes'
            movie_value_dict['movie_title_xs'] = movie_title_xs #global movie_title_xs
            movie_value_dict['possible_title_en'] = possible_title_en_list
            movie_value_dict['href'] = possible_href_list ##            
            movies_reviews_dict[movie_title] = movie_value_dict
            print('movie_title: %s, more_results_counter: %s'%(movie_title, more_results_counter))

    else:
        global no_data_counter
        no_data_counter += 1
        
        movie_value_dict['memo'] = 'no data @atmovies'
        movies_reviews_dict[movie_title] = movie_value_dict
        print('movie_title: %s, no_data_counter: %s'%(movie_title, no_data_counter))



    
if __name__ == "__main__":
    

    numThread = 100
    threads = ThreadPoolExecutor(numThread)
    futures = []
    thStart = datetime.now()

    with ThreadPoolExecutor(max_workers=numThread) as executor:
    
        with open('/home/ubuntu/Desktop/movies_title.json') as f:
            movies_title = json.loads(f.read())
            for j in range(len(movies_title)):
                movie_title = movies_title[j]
                futures.append(threads.submit(movies_crawler, movie_title)) #查詢結果不在一起不ok
    
    wait(futures)
    thEnd = datetime.now()
    timeSpent = str(thEnd - thStart).split('.')[0]
    print("執行緒:" + str(numThread))    
    print("耗時:" + timeSpent)

    
    
    #記得打開！！！
    with open('movies_title_ch_backup_%s_v2.json'%today, 'w') as f:
        f.write(json.dumps(movies_reviews_dict, ensure_ascii=False, indent=4))
    
    print('\n' + 'REPORT:')
    print('total_counter: %s'%total_counter)
    print('\n')
    print('one_result_counter: %s'%one_result_counter)
    print('perfectly_matched_result_counter: %s'%perfectly_matched_result_counter)
    print('no_movie_data_counter: %s'%no_movie_data_counter)
    print('more_results_counter: %s'%more_results_counter)
    print('no_data_counter: %s'%no_data_counter)
    print('\n' + 'movies_reviews_dict:')
    

movie_title: Star Wars: Episode VII - The Force Awakens, one_result_counter: 1
movie_title: The Lord of the Rings: The Return of the King, perfectly_matched_result_counter: 1
movie_title: Jurassic World, perfectly_matched_result_counter: 2
movie_title: The Fate of the Furious, one_result_counter: 2
movie_title: Skyfall, perfectly_matched_result_counter: 3
movie_title: The Dark Knight Rises, one_result_counter: 3
movie_title: Avatar, perfectly_matched_result_counter: 4
movie_title: Rogue One, one_result_counter: 4
movie_title: Pirates of the Caribbean: On Stranger Tides, perfectly_matched_result_counter: 5
movie_title: Titanic, perfectly_matched_result_counter: 6
movie_title: Star Wars: Episode I - The Phantom Menace, one_result_counter: 5
movie_title: Harry Potter and the Sorcerer's Stone, no_movie_data_counter: 1
movie_title: Alice in Wonderland, perfectly_matched_result_counter: 7
movie_title: Harry Potter and the Half-Blood Prince, no_movie_data_counter: 2
movie_title: Finding Nemo,

movie_title: Iron Man, perfectly_matched_result_counter: 76
movie_title: American Sniper, perfectly_matched_result_counter: 77
movie_title: Sherlock Holmes: A Game of Shadows, perfectly_matched_result_counter: 78
movie_title: The Day After Tomorrow, perfectly_matched_result_counter: 79
movie_title: X-Men: Apocalypse, one_result_counter: 31
movie_title: King Kong, perfectly_matched_result_counter: 80
movie_title: Star Wars: Episode V - The Empire Strikes Back, one_result_counter: 32
movie_title: World War Z, perfectly_matched_result_counter: 81
movie_title: Despicable Me, perfectly_matched_result_counter: 82
movie_title: Mission: Impossible II, no_data_counter: 11
movie_title: Ted, perfectly_matched_result_counter: 83
movie_title: The Revenant, one_result_counter: 33
movie_title: The Simpsons Movie, perfectly_matched_result_counter: 84
movie_title: Home Alone, perfectly_matched_result_counter: 85
movie_title: Dunkirk, one_result_counter: 34
movie_title: Terminator 2: Judgment Day, one_r

movie_title: Kimi no na wa., one_result_counter: 57
movie_title: Top Gun, perfectly_matched_result_counter: 169
movie_title: The Legend of Tarzan, perfectly_matched_result_counter: 170
movie_title: American Beauty, perfectly_matched_result_counter: 171
movie_title: Die Hard with a Vengeance, no_data_counter: 14
movie_title: Apollo 13, perfectly_matched_result_counter: 172movie_title: Hotel Transylvania, perfectly_matched_result_counter: 173

movie_title: Rain Man, perfectly_matched_result_counter: 174
movie_title: Basic Instinct, perfectly_matched_result_counter: 175
movie_title: Who Framed Roger Rabbit, one_result_counter: 58
movie_title: Dinosaur, perfectly_matched_result_counter: 176
movie_title: Kingsman: The Golden Circle, one_result_counter: 59
movie_title: Hannibal, perfectly_matched_result_counter: 177
movie_title: Deep Impact, one_result_counter: 60
movie_title: The Lorax, one_result_counter: 61
movie_title: The Hangover Part III, no_data_counter: 15
movie_title: The Great Gat

movie_title: Batman Returns, one_result_counter: 82movie_title: Lethal Weapon 4, perfectly_matched_result_counter: 259

movie_title: Home, perfectly_matched_result_counter: 260
movie_title: Oblivion, perfectly_matched_result_counter: 261
movie_title: Superman, perfectly_matched_result_counter: 262
movie_title: Turbo, perfectly_matched_result_counter: 263
movie_title: Mr. Peabody & Sherman, perfectly_matched_result_counter: 264
movie_title: The Tourist, perfectly_matched_result_counter: 265movie_title: The Bourne Legacy, one_result_counter: 84movie_title: Sen to Chihiro no kamikakushi, one_result_counter: 83


movie_title: Jerry Maguire, perfectly_matched_result_counter: 266
movie_title: Scary Movie, perfectly_matched_result_counter: 267
movie_title: The Silence of the Lambs, perfectly_matched_result_counter: 268
movie_title: Lilo & Stitch, perfectly_matched_result_counter: 269
movie_title: Lincoln, perfectly_matched_result_counter: 270
movie_title: Rocky III, one_result_counter: 85movi

movie_title: Wild Wild West, perfectly_matched_result_counter: 350
movie_title: Presumed Innocent, one_result_counter: 111
movie_title: The Bourne Supremacy, one_result_counter: 112
movie_title: Inferno, perfectly_matched_result_counter: 351
movie_title: Elf, perfectly_matched_result_counter: 352
movie_title: What Happens in Vegas, perfectly_matched_result_counter: 353
movie_title: Interview with the Vampire: The Vampire Chronicles, no_data_counter: 34
movie_title: Rocky, perfectly_matched_result_counter: 354
movie_title: Captain Phillips, perfectly_matched_result_counter: 355
movie_title: The Prince of Egypt, perfectly_matched_result_counter: 356movie_title: Central Intelligence, one_result_counter: 113

movie_title: Bedtime Stories, perfectly_matched_result_counter: 357
movie_title: Scary Movie 3, perfectly_matched_result_counter: 358
movie_title: Jack Reacher, perfectly_matched_result_counter: 359
movie_title: Journey to the West: Demon Chapter, one_result_counter: 114
movie_title: 

...

movie_title: Flex is Kings, one_result_counter: 6679
movie_title: Kelly & Cal, one_result_counter: 6680
movie_title: The Only Real Game, one_result_counter: 6681
movie_title: A Picture of You, perfectly_matched_result_counter: 4015
movie_title: The Giant Mechanical Man, one_result_counter: 6682
movie_title: Watercolors, one_result_counter: 6683
movie_title: One Small Hitch, one_result_counter: 6684
movie_title: Misery Loves Comedy, one_result_counter: 6685movie_title: Choose Connor, one_result_counter: 6686

movie_title: La question humaine, one_result_counter: 6687
movie_title: Schaste moe, no_data_counter: 3292
movie_title: Violet Tendencies, one_result_counter: 6688
movie_title: The Amazing Truth About Queen Raquela, one_result_counter: 6689
movie_title: De Mayerling à Sarajevo, one_result_counter: 6690
movie_title: Land of the Blind, perfectly_matched_result_counter: 4016
movie_title: Nostalghia, one_result_counter: 6691
movie_title: Khon fai bin, one_result_counter: 6692
movie_tit

movie_title: Contracted, perfectly_matched_result_counter: 4031
movie_title: Auto Shop of Horrors, one_result_counter: 6760
movie_title: Yaad Rayaad, no_data_counter: 3337
movie_title: This Divided State, no_data_counter: 3338movie_title: Mr. Love, no_data_counter: 3339

movie_title: The House of Magic, one_result_counter: 6761
movie_title: Silver Circle, one_result_counter: 6762
movie_title: Steve McQueen: The Man & Le Mans, one_result_counter: 6763
movie_title: Inescapable, one_result_counter: 6764
movie_title: Outta Time, one_result_counter: 6765
movie_title: Swim Team, one_result_counter: 6766
movie_title: Couleur de peau: Miel, one_result_counter: 6767
movie_title: Lucky Numbers, perfectly_matched_result_counter: 4032
movie_title: Défense d'aimer, no_data_counter: 3340
movie_title: Coming Through the Rye, one_result_counter: 6768
movie_title: DJ Stan Da Man, no_data_counter: 3341
movie_title: Dream Boat, one_result_counter: 6769
movie_title: Ultrasuede: In Search of Halston, one_r

movie_title: Protektor, one_result_counter: 6835
movie_title: Fast Talking, no_data_counter: 3375movie_title: Brown's Requiem, no_data_counter: 3376

movie_title: Equal Strength, no_data_counter: 3377
movie_title: George Bush Goes to Heaven, no_data_counter: 3378movie_title: Storefront Hitchcock, no_data_counter: 3379

movie_title: Vamps, perfectly_matched_result_counter: 4059
movie_title: The Hacker Wars, one_result_counter: 6836
movie_title: Fremde Haut, no_data_counter: 3380
movie_title: Poklosie, one_result_counter: 6837
movie_title: Ash Wednesday, one_result_counter: 6838
movie_title: Members Only, one_result_counter: 6839
movie_title: Nights and Weekends, one_result_counter: 6840
movie_title: The Professor: Tai Chi's Journey West, one_result_counter: 6841
movie_title: The Ministers, one_result_counter: 6842
movie_title: 16 Heads and Counting, no_data_counter: 3381
movie_title: The Girl on the Train, perfectly_matched_result_counter: 4060
movie_title: Wheels of Fate, no_data_count

movie_title: Miles, perfectly_matched_result_counter: 4082
movie_title: Cheerful Weather for the Wedding, one_result_counter: 6902
movie_title: The Navigators, one_result_counter: 6903
movie_title: Deadlands: The Rising, no_data_counter: 3425
movie_title: Trial by Combat, no_data_counter: 3426
movie_title: The Reconstruction of William Zero, no_data_counter: 3427
movie_title: Sneakerheadz, one_result_counter: 6904
movie_title: Among the Believers, one_result_counter: 6905
movie_title: Cold Winter's Night, no_data_counter: 3428
movie_title: Catch .44, one_result_counter: 6906
movie_title: Fanny, perfectly_matched_result_counter: 4083
movie_title: Homo Sapiens 1900, no_data_counter: 3429
movie_title: Shrooms, one_result_counter: 6907
movie_title: Best Man Down, one_result_counter: 6908
movie_title: American Strays, no_data_counter: 3430
movie_title: Up for Grabs, no_data_counter: 3431
movie_title: A Kind of Murder, one_result_counter: 6909
movie_title: Everly, perfectly_matched_result_co

movie_title: Young at Hearts, no_data_counter: 3488
movie_title: Abs of Steel, no_data_counter: 3489
movie_title: La león, one_result_counter: 6961
movie_title: The Dog Problem, one_result_counter: 6962
movie_title: Four Lane Highway, no_data_counter: 3490
movie_title: Rhymes for Young Ghouls, one_result_counter: 6963
movie_title: The Strategist Anthology, no_data_counter: 3491
movie_title: She's Lost Control, one_result_counter: 6964
movie_title: Sweet Jane, no_data_counter: 3492
movie_title: Eastside, no_data_counter: 3493
movie_title: Ekho: Fall of an Empire, no_data_counter: 3494movie_title: Vietnam Long Time Coming, no_data_counter: 3495

movie_title: Au delà de la haine, no_data_counter: 3496
movie_title: Flaming Star, perfectly_matched_result_counter: 4103
movie_title: How to Be a Serial Killer, perfectly_matched_result_counter: 4104
movie_title: Wishman, no_data_counter: 3497
movie_title: Superstar Female Serial Killer, no_data_counter: 3498
movie_title: Perrier's Bounty, one_r